### Blocks World

An extension of blocks world with two agents.
- one can only moves blocks marked with vowels, the other with consonants.

The goal is to arrange the blocks according to the goal.

In [8]:
from src.llm_plan.LLM import GPT_Ollama
from src.llm_plan.Problem import ProblemStaticBlocksworld
from src.llm_plan.StaticEnvironment import StaticBlocksworld

model = GPT_Ollama()
env = StaticBlocksworld(num_vowels=3, num_consonants=3, easy=True)
problem = ProblemStaticBlocksworld(env)

env.render()

I    Q    W    E    A    R


In [9]:
# Generate the PDDL plans for each agent
agents = ["Agent A", "Agent B"]
agent_prompts = {agent: "" for agent in agents}
agent_responses = {agent: "" for agent in agents}

for agent in agents:
    print(f"Prompting {model.model_name} with the following prompt:\n")
    print(problem.prompts[agent])
    agent_prompts[agent] = f"{agent} Problem:\n" + problem.prompts[agent] + "\n"
    answer = model.generate_sync(problem.system_prompts[agent], 
                                 problem.prompts[agent])
    print("Answer:")
    print(answer)
    agent_responses[agent] = f"{agent} Response:\n" + answer + "\n"
    print("-"*30 + "\n")

Prompting gpt-oss-20B-ollama with the following prompt:

You are Agent A. You are in an enviroment with the following public information:
I am playing with a set of blocks where I need to arrange the blocks into stacks.I can perform the following actions: pick up a block; unstack a block from on topof another block; put down a block; stack a block on top of another block.
I can only pick up or unstack one block at a time.
I can only pick up or unstack a block if my hand is empty.
I can only pick up a block if the block is on the table and the block is clear. A block is clear if the block has no otherblocks on top of it and if the block is not picked up.
I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block.
I can only unstack a block from on top of another block if the block I am unstacking is clear.
Once I pick up or unstack a block, I am holding the block.
I can only put down a block that I am holding.
I can only sta

In [10]:
# Prompt the orchestrator for the final plan
orchestrator_prompt = problem.prompts["Orchestrator"].format(pddl_agent_A=agent_prompts["Agent A"] + agent_responses["Agent A"],
                                                             pddl_agent_B=agent_prompts["Agent B"] + agent_responses["Agent B"],
                                                             goal=env.goal["Agent A"])

final_plan = model.generate_sync(problem.system_prompts["Orchestrator"], 
                                 orchestrator_prompt)

print(final_plan)                       

Below is a **single PDDL domain** that contains the two agents in the same world.  
The domain uses two separate hands (`hand‑empty‑a` / `hand‑empty‑b`) and two
separate *holding* predicates (`holding‑a` / `holding‑b`).  
Blocks that are vowels can only be moved by agent *A*; blocks that are
consonants can only be moved by agent *B*.  The world objects
(`on`, `on-table`, `clear`, `vowel`, `consonant`) are shared by both agents.

```pddl
;; ==============================================================
;; Domain:  dual‑agent‑block‑world
;; ==============================================================

(define (domain dual-agent-block-world)
  (:requirements :typing :strips)

  (:types block)

  (:predicates
    ;; world predicates
    (on          ?b1 ?b2 - block)   ; ?b1 is on top of ?b2
    (on-table    ?b - block)        ; ?b is directly on the table
    (clear       ?b - block)        ; nothing on top of ?b
    (vowel       ?b - block)        ; block is a vowel
    (consonant   ?b 

In [11]:
# Isolate the pddl problem and domain
pddl_domain = model.generate_sync("You are a helpful assistant with expertise with PDDL.", 
                                  "Extract the PDDL domain from this text:\n" + final_plan + "\n Just give me the domain, nothing else. Avoid any comments or explanations.")

pddl_problem = model.generate_sync("You are a helpful assistant with expertise with PDDL.", 
                                   "Extract the PDDL problem from this text:\n" + final_plan + "\n Just give me the problem, nothing else. Avoid any comments or explanations.")

In [12]:
# Save the PDDL domain and problem to files
with open("./tmp/domain.pddl", "w") as domain_file:
    domain_file.write(pddl_domain)

with open("./tmp/problem.pddl", "w") as problem_file:
    problem_file.write(pddl_problem)


In [13]:
import subprocess

# Invoke fast downward and solve the problem
command = "./solvers/fast-downward-24.06.1/fast-downward.py --alias lama-first --plan-file ./tmp/sas_plan ./tmp/domain.pddl ./tmp/problem.pddl > ./tmp/logs.txt 2>&1"

subprocess.run(command, shell=True)

CompletedProcess(args='./solvers/fast-downward-24.06.1/fast-downward.py --alias lama-first --plan-file ./tmp/sas_plan ./tmp/domain.pddl ./tmp/problem.pddl > ./tmp/logs.txt 2>&1', returncode=12)